# Data Savers

In this tutorial, we will explore the examples of how the data could be saved in the HDF5 format using the Libra's `data_savers` module

In [40]:
import sys
import cmath
import math
import os
import h5py
import time
import matplotlib.pyplot as plt   # plots
import numpy as np
#from matplotlib.mlab import griddata
%matplotlib inline 


if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *

from libra_py import units
import libra_py.models.Tully as Tully
from libra_py import tsh
from libra_py import tsh_stat
from libra_py import data_conv
from libra_py import data_savers
from libra_py import dynamics_plotting
from libra_py import dynamics_exact
import util.libutil as comn


plt.rc('axes', titlesize=24)      # fontsize of the axes title
plt.rc('axes', labelsize=20)      # fontsize of the x and y labels
plt.rc('legend', fontsize=20)     # legend fontsize
plt.rc('xtick', labelsize=16)    # fontsize of the tick labels
plt.rc('ytick', labelsize=16)    # fontsize of the tick labels

plt.rc('figure.subplot', left=0.2)
plt.rc('figure.subplot', right=0.95)
plt.rc('figure.subplot', bottom=0.13)
plt.rc('figure.subplot', top=0.88)

colors = {}

colors.update({"11": "#8b1a0e"})  # red       
colors.update({"12": "#FF4500"})  # orangered 
colors.update({"13": "#B22222"})  # firebrick 
colors.update({"14": "#DC143C"})  # crimson   

colors.update({"21": "#5e9c36"})  # green
colors.update({"22": "#006400"})  # darkgreen  
colors.update({"23": "#228B22"})  # forestgreen
colors.update({"24": "#808000"})  # olive      

colors.update({"31": "#8A2BE2"})  # blueviolet
colors.update({"32": "#00008B"})  # darkblue  

colors.update({"41": "#2F4F4F"})  # darkslategray

clrs_index = ["11", "21", "31", "41", "12", "22", "32", "13","23", "14", "24"]

In [41]:
class mem_saver:
    """
    This class is needed for saving variables into a dictionary

    Example of usage:
    
        x = mem_saver(["q", "p"])

        x.add_data("q", 1.0)
        x.add_data("q", -1.0)

        print(x.data["q"])

    """


    def __init__(self, _keywords=[]):
        """
        Initializes an object that would store the data
        in different formats - the temporary and the numpy-consistent
        
        """
        
        # Names of the data sets
        self.keywords = list(_keywords)  
        
        # "Raw" data - elements could be of whatever data type
        self.data = {}  
        
        # "Numpy" data - elements are numpy arrays
        self.np_data = {}
        
        
        # Only initialize the "raw" data, don't touch the numpy
        for keyword in self.keywords:
            self.data[keyword] = []            

            
    def add_data(self, data_name, _data):
        """
        To collect arbitrary data
        
        Args:
            data_name (string): the name of the data set
            _data (anything): the data to be added
        
        
        This function simply appends the data elements
        to the prepared lists. There is no restriction on the 
        data type for the elements added
        """
        if keyword in self.keywords:
            self.data[data_name].append(_data)

                        
    def add_dataset(self, data_name, shape, dtype):
        """
        To initialize the numpy arrays to hold the data
        
        Args:
            data_name (string): the name of the data set
            shape (tuple of ints): the dimensions of the numpy array
            dtype (string): ["I", "R", or "C"] - the tye of data to be stored in the array
            
        """
        if data_name in self.keywords:
            pass
        else:
            self.keywords.append(data_name)
                
        if dtype=="I":
            self.np_data[data_name] = np.empty(shape, int)
            
        elif dtype=="R":
            self.np_data[data_name] = np.empty(shape, float)            
            
        elif dtype=="C":
            self.np_data[data_name] = np.empty(shape, complex)            
            
        else:
            print(F"ERROR: the dtype = {dtype} is not allowed in add_dataset")
        
    
              
    def save_scalar(self, istep, data_name, _data):
        """
        Saves a scalar to 1D array
        """

        if data_name in self.keywords:
            self.np_data[data_name][istep] = _data
            
            
    def save_multi_scalar(self, istep, iscal, data_name, _data):
        """
        Saves a sacalar to 2D array
        """

        if data_name in self.keywords:            
            self.np_data[data_name][istep, iscal] = _data

        
    def save_matrix(self, istep, data_name, _data):
        """          
        Saves a matrix to a 3D array
        
        Args:

          istep ( int ) :  index of the timestep for the data
          data_name ( string ) : how to call this data set internally
          _data ( (C)MATRIX(nx, ny) ) : the actual data to save          
           
        """

        if data_name in self.keywords:

            nx, ny = _data.num_of_rows, _data.num_of_cols

            for i in range(nx):
                for j in range(ny):
                    self.np_data[data_name][istep, i, j] = _data.get(i, j)


    def save_multi_matrix(self, istep, imatrix, data_name, _data):
        """          
        Saves one of a series of matrices

        Args:
        
          istep ( int ) :  index of the timestep for the data
          data_name ( string ) : how to call this data set internally
          data ( (C)MATRIX(nx, ny) ) : the actual data to save
           
        """

        if data_name in self.keywords:

            nx, ny = _data.num_of_rows, _data.num_of_cols

            for i in range(nx):
                for j in range(ny):
                    self.np_data[data_name][istep, imatrix, i, j] = _data.get(i, j)
                        

    
    def save_data(self, filename, data_names, mode):
        """
        To save the numpy data into HDF5 files
        
        Args:
            filename (string): the name of the HDF5 file where to save the res
            data_names (list of strings): the list of the names of the data sets to save
            mode ("w" or "a"): whether to overwrite the file or to append to it
        
        """
        
        with h5py.File(filename, mode) as f:
            
            for data_name in data_names:
                
                if data_name in self.keywords:
                                                        
                    g = f.create_group(data_name)                                                            
                    g.create_dataset("data", data = self.np_data[data_name])
                    
                                        
    


In [43]:
N = 10
saver = mem_saver()
saver.add_dataset("q", (N,N), float)
saver.add_dataset("p", (N,N), float)

for i in range(N):
    for j in range(N):
        saver.save_multi_scalar(i,j, "q", i-j)
        saver.save_multi_scalar(i,j, "p", i+j)
        #saver.np_data["q"][i,j] = i - j
        #saver.np_data["p"][i,j] = i + j


print(saver.np_data["q"])
print(saver.np_data["q"].shape)
print(saver.np_data["q"].dtype)
print(saver.keywords)

saver.save_data("test.hdf5", ["q"], "w" )
saver.save_data("test.hdf5", ["p"], "a" )


[[ 0. -1. -2. -3. -4. -5. -6. -7. -8. -9.]
 [ 1.  0. -1. -2. -3. -4. -5. -6. -7. -8.]
 [ 2.  1.  0. -1. -2. -3. -4. -5. -6. -7.]
 [ 3.  2.  1.  0. -1. -2. -3. -4. -5. -6.]
 [ 4.  3.  2.  1.  0. -1. -2. -3. -4. -5.]
 [ 5.  4.  3.  2.  1.  0. -1. -2. -3. -4.]
 [ 6.  5.  4.  3.  2.  1.  0. -1. -2. -3.]
 [ 7.  6.  5.  4.  3.  2.  1.  0. -1. -2.]
 [ 8.  7.  6.  5.  4.  3.  2.  1.  0. -1.]
 [ 9.  8.  7.  6.  5.  4.  3.  2.  1.  0.]]
(10, 10)
float64
['q', 'p']


In [44]:
with h5py.File("test.hdf5", "r") as f:    
    print(f["q/data"][:,:])    
    print(f["p/data"][:,:])
            


[[ 0. -1. -2. -3. -4. -5. -6. -7. -8. -9.]
 [ 1.  0. -1. -2. -3. -4. -5. -6. -7. -8.]
 [ 2.  1.  0. -1. -2. -3. -4. -5. -6. -7.]
 [ 3.  2.  1.  0. -1. -2. -3. -4. -5. -6.]
 [ 4.  3.  2.  1.  0. -1. -2. -3. -4. -5.]
 [ 5.  4.  3.  2.  1.  0. -1. -2. -3. -4.]
 [ 6.  5.  4.  3.  2.  1.  0. -1. -2. -3.]
 [ 7.  6.  5.  4.  3.  2.  1.  0. -1. -2.]
 [ 8.  7.  6.  5.  4.  3.  2.  1.  0. -1.]
 [ 9.  8.  7.  6.  5.  4.  3.  2.  1.  0.]]
[[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
 [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
 [ 2.  3.  4.  5.  6.  7.  8.  9. 10. 11.]
 [ 3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
 [ 4.  5.  6.  7.  8.  9. 10. 11. 12. 13.]
 [ 5.  6.  7.  8.  9. 10. 11. 12. 13. 14.]
 [ 6.  7.  8.  9. 10. 11. 12. 13. 14. 15.]
 [ 7.  8.  9. 10. 11. 12. 13. 14. 15. 16.]
 [ 8.  9. 10. 11. 12. 13. 14. 15. 16. 17.]
 [ 9. 10. 11. 12. 13. 14. 15. 16. 17. 18.]]


In [45]:
nsteps = 10
ntraj = 5
nstates = 2

saver = mem_saver()
saver.add_dataset("Hvib", (nsteps, ntraj, nstates, nstates), complex)

for step in range(nsteps):
    for traj in range(ntraj):
        H = CMATRIX(nstates,nstates)
        H.set(0,0, step*(1.0+0.0j))
        H.set(1,1, traj*(0.0+1.0j))
        
        saver.save_multi_matrix(step, traj, "Hvib", H )

saver.save_data("test2.hdf5", ["Hvib"], "w")

In [46]:
with h5py.File("test2.hdf5", "r") as f:    
    print(f["Hvib/data"][:,:, 0, 0])        
    print(f["Hvib/data"][:,:, 1, 1])    
    

[[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j]
 [2.+0.j 2.+0.j 2.+0.j 2.+0.j 2.+0.j]
 [3.+0.j 3.+0.j 3.+0.j 3.+0.j 3.+0.j]
 [4.+0.j 4.+0.j 4.+0.j 4.+0.j 4.+0.j]
 [5.+0.j 5.+0.j 5.+0.j 5.+0.j 5.+0.j]
 [6.+0.j 6.+0.j 6.+0.j 6.+0.j 6.+0.j]
 [7.+0.j 7.+0.j 7.+0.j 7.+0.j 7.+0.j]
 [8.+0.j 8.+0.j 8.+0.j 8.+0.j 8.+0.j]
 [9.+0.j 9.+0.j 9.+0.j 9.+0.j 9.+0.j]]
[[0.+0.j 0.+1.j 0.+2.j 0.+3.j 0.+4.j]
 [0.+0.j 0.+1.j 0.+2.j 0.+3.j 0.+4.j]
 [0.+0.j 0.+1.j 0.+2.j 0.+3.j 0.+4.j]
 [0.+0.j 0.+1.j 0.+2.j 0.+3.j 0.+4.j]
 [0.+0.j 0.+1.j 0.+2.j 0.+3.j 0.+4.j]
 [0.+0.j 0.+1.j 0.+2.j 0.+3.j 0.+4.j]
 [0.+0.j 0.+1.j 0.+2.j 0.+3.j 0.+4.j]
 [0.+0.j 0.+1.j 0.+2.j 0.+3.j 0.+4.j]
 [0.+0.j 0.+1.j 0.+2.j 0.+3.j 0.+4.j]
 [0.+0.j 0.+1.j 0.+2.j 0.+3.j 0.+4.j]]
